In [ ]:
from pathlib import Path
import json
import re

trackDataDict = json.loads(Path("tracks.js.bak").read_text().replace("var tracks = ", ""))

In [ ]:
def fixOctave(inputNote, inputOctave):
    if inputNote in ["A", "A♯", "B"]:
        return inputOctave-1
    else:
        return inputOctave

In [ ]:
fixed_dict = {}

for track, data in trackDataDict.items():
    tempNotes = data.get("notes", [])
    tempDoubleNotes = data.get("doublenotes", [])
    tempHalfNotes = data.get("halfnotes", [])

    tempNotes = [
        [f"{note}{fixOctave(note, octave)}", duration]
        for inputNote, duration in tempNotes
        if (octave := int(inputNote[-1:])) and (note := inputNote[:-1])
    ]
    
    tempDoubleNotes = [
        [f"{note}{fixOctave(note, octave)}", duration]
        for inputNote, duration in tempDoubleNotes
        if (octave := int(inputNote[-1:])) and (note := inputNote[:-1])
    ]
    
    tempHalfNotes = [
        [f"{note}{fixOctave(note, octave)}", duration, pattern]
        for inputNote, duration, pattern in tempHalfNotes
        if (octave := int(inputNote[-1:])) and (note := inputNote[:-1])
    ]
    
    if len(tempNotes):
        data["notes"] = tempNotes
        
    if len(tempDoubleNotes):
        data["doublenotes"] = tempDoubleNotes
        
    if len(tempHalfNotes):
        data["halfnotes"] = tempHalfNotes
        
    ## get the highest and lowest octaves
    try:
        maxOct = max(map(lambda x: int(x[0][-1:]), tempNotes + tempDoubleNotes + tempHalfNotes))
        minOct = min(map(lambda x: int(x[0][-1:]), tempNotes + tempDoubleNotes + tempHalfNotes))
        deltaOct = maxOct - minOct
        ## fix octave metadata
        data["octaves"] = deltaOct + 1
        data["startOctave"] = minOct
    except:
        pass
    
    fixed_dict[track] = data

In [ ]:
fixedJson = json.dumps(fixed_dict, indent=4).replace(r"\u266f", "♯")

fixedJson = re.sub(r'\[\n +', '[', fixedJson)
fixedJson = re.sub(r'\],\n +', '],', fixedJson)
fixedJson = re.sub(r'",\n +', '",', fixedJson)
fixedJson = re.sub(r'\n +\]', ']', fixedJson)
fixedJson = re.sub(r'"halfnotes"', '\n        "halfnotes"', fixedJson)
fixedJson = fixedJson.replace(r'\u266f', '♯')
fixedJson = re.sub(r',\n                "', ',"', fixedJson)
fixedJson = re.sub(r',"doublenotes"', ',\n        "doublenotes"', fixedJson)

In [ ]:
with open("tracks.js", "w") as outfile:
    outfile.write("var tracks = ")
    outfile.write(fixedJson)